## IMPORTS

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]= "0" #imports
#imports
import torch
import torch.nn.functional as F
%config Completer.use_jedi = False

%env CUDA_LAUNCH_BLOCKING 1

/mnt/scratch/lts2/karalias/envs/nfe/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


env: CUDA_LAUNCH_BLOCKING=1


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

/tmp/ipykernel_1040195/495279268.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [12]:
from matplotlib import pyplot as plt
from torch_geometric.utils.convert import from_networkx
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import torch_geometric
from torch_geometric.data import DataLoader, DenseDataLoader as DenseLoader
from torch_geometric.utils import softmax, add_self_loops, remove_self_loops, segregate_self_loops, remove_isolated_nodes, contains_isolated_nodes, add_remaining_self_loops
from torch_geometric.utils import dropout_adj, to_undirected, to_networkx
from torch_geometric.utils import is_undirected
from torch_geometric.datasets import TUDataset
from networkx.algorithms.approximation import max_clique
from torch_geometric.data import DataListLoader
from networkx.algorithms.approximation import max_clique
from networkx.algorithms import graph_clique_number
from networkx.algorithms import find_cliques

In [13]:
#Solve using a networkx object
def solve_gurobi_mis(nx_graph, costs=None, time_limit = None):

    x_vars = {}
    c_vars = {}
    m = gp.Model("mip1")
    m.params.OutputFlag = 0

    if time_limit:
        m.params.TimeLimit = time_limit

    for node in nx_graph.nodes():
        x_vars['x_'+str(node)] = m.addVar(vtype=GRB.BINARY, name="x_"+str(node))
    

    count_edges = 0
    for edge in nx_graph.edges():
        m.addConstr(x_vars['x_'+str(edge[0])] + x_vars['x_'+str(edge[1])] <= 1,'c_'+str(count_edges))
        count_edges+=1
    if costs:
        m.setObjective(sum([x_vars['x_'+str(node)]*costs[node] for node in nx_graph.nodes()]), GRB.MAXIMIZE)
    else:
        m.setObjective(sum([x_vars['x_'+str(node)] for node in nx_graph.nodes()]), GRB.MAXIMIZE)



    # Optimize model
    m.optimize();

    set_size = m.objVal;
    x_vals = [var.x for var in m.getVars()] 

    return set_size, x_vals

In [14]:
def solve_gurobi_maxclique(nx_graph, costs = None, time_limit = None):
    nx_complement = nx.operators.complement(nx_graph)
    nx_graph = nx_complement
    x_vars = {}
    c_vars = {}
    m = gp.Model("mip1")
    m.params.OutputFlag = 0

    if time_limit:
        m.params.TimeLimit = time_limit

    for node in nx_graph.nodes():
        x_vars['x_'+str(node)] = m.addVar(vtype=GRB.BINARY, name="x_"+str(node))
  


    count_edges = 0
    for edge in nx_graph.edges():
        m.addConstr(x_vars['x_'+str(edge[0])] + x_vars['x_'+str(edge[1])] <= 1,'c_'+str(count_edges))
        count_edges+=1
    
    if costs:
        m.setObjective(sum([x_vars['x_'+str(node)]*costs[node] for node in nx_graph.nodes()]), GRB.MAXIMIZE)
    else:
        m.setObjective(sum([x_vars['x_'+str(node)] for node in nx_graph.nodes()]), GRB.MAXIMIZE)




    # Optimize model
    m.optimize();

    set_size = m.objVal;
    x_vals = [var.x for var in m.getVars()] 

    return set_size, x_vals

In [15]:
def solve_gurobi_minvertcover(nx_graph, time_limit = None):
    nx_complement = nx.operators.complement(nx_graph)
    x_vars = {}
    m = gp.Model("mip1")
    m.params.OutputFlag=0

    if time_limit:
        m.params.TimeLimit = time_limit

    for node in nx_complement.nodes():
        x_vars['x_'+str(node)] = m.addVar(vtype=GRB.BINARY, name="x_"+str(node))

    count_edges = 0
    for edge in nx_complement.edges():
        m.addConstr(x_vars['x_'+str(edge[0])] + x_vars['x_'+str(edge[1])] >= 1,'c_'+str(count_edges))
        count_edges+=1
    m.setObjective(sum([x_vars['x_'+str(node)] for node in nx_complement.nodes()]), GRB.MINIMIZE);

    # Optimize model
    m.optimize();

    set_size = m.objVal;
    x_vals = [var.x for var in m.getVars()] 

    return set_size, x_vals

In [16]:
#Solve directly from pytorch geometric object
def solve_gurobi_maxcut(pyg_graph_data, time_limit = None):
    # Create a new model

    m = gp.Model("maxcut")
    m.params.OutputFlag = 0

    # time limit in seconds, if applicable
    if time_limit:
        m.params.TimeLimit = time_limit
    
    # Set up node variables
    x_vars = {}
    for i in range(pyg_graph_data.num_nodes):
        x_vars["x_" + str(i)] = m.addVar(vtype=GRB.BINARY, name="x_" + str(i))

    r,c = pyg_graph_data.edge_index
    # Set objective
    obj = gp.QuadExpr()
    #Iterate over edges to compute (x_i - x_j)**2 for each edge (i,j) and sum it all up
    for source, target in zip(r,c):
        qi_qj = (x_vars['x_' + str(source.item())] - x_vars['x_' + str(target.item())])
        obj += qi_qj * qi_qj / 2
    m.setObjective(obj, GRB.MAXIMIZE)

    # Optimize model
    m.optimize()

    set_size = m.objVal
    x_vals = [var.x for var in m.getVars()] 

    return set_size, x_vals

## Maxcut example

In [17]:
  #Load  dataset
dataset = TUDataset(root='/tmp/'+'PROTEINS', name='PROTEINS')
dataset = list(dataset)
mini_dataset = dataset[:10]

In [18]:
#get gurobi maxcuts (only for the first few graphs as an example)
for data in mini_dataset:
    r,c = data.edge_index
    cut_size, optimal_set = solve_gurobi_maxcut(data)
    vertcover_size, optimal_set = solve_gurobi_minvertcover(to_networkx(data))
    clique_size, optimal_set = solve_gurobi_maxclique(to_networkx(data))
    mis_size, optimal_set = solve_gurobi_mis(to_networkx(data), None)
    data.max_cut = cut_size
    data.min_vertcover = vertcover_size
    data.max_clique = clique_size
    data.mis_size = mis_size